Install and set up Airflow: The first step is to install Apache Airflow on your local machine or a remote server. This can be done by running the command pip install apache-airflow. Once Airflow is installed, you'll need to initialize the database and start the web server. This can be done by running the command airflow initdb and airflow webserver.

In [1]:
!pip install apache-airflow


Defaulting to user installation because normal site-packages is not writeable

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for cron-descriptor: filename=cron_descriptor-1.2.32-py3-none-any.whl size=46747 sha256=8a39cf4b1be5af66ef0b64087de8383107528b7f053012a5d7af9fd7cb0dc92d
  Stored in directory: c:\users\tinul piumika\appdata\local\pip\cache\wheels\62\6f\ad\ac401d9d574d2c272e1726b78874173d975af721fd54655175
  Created wheel for pendulum: filename=pendulum-2.1.2-cp39-cp39-win_amd64.whl size=109762 sha256=92975dcee2062191f71c736fbceb81f7dbc3d9e6ca329b67f21581b547b5974a
  Stored in directory: c:\users\tinul piumika\appdata\local\pip\cache\wheels\c9\c7\29\3cf0033406b18958d1a7b107c38be2405ef9c081b64a264fd5
  Created wheel for python-nvd3: filename=python_nvd3-0.15.0-py3-none-any.whl size

You'll also need to initialize the Airflow database by running:

In [2]:
!airflow initdb


'airflow' is not recognized as an internal or external command,
operable program or batch file.


And start the airflow web server and scheduler:

In [ ]:
airflow webserver -p 8080
airflow scheduler


Create an ETL DAG: Once Airflow is set up, you'll need to create a DAG (Directed Acyclic Graph) to define your ETL workflow. A DAG is a collection of tasks that run to completion. For example, here's a simple DAG that reads data from a CSV file, processes it, and writes it to a database:

In [ ]:
# #prev
# from airflow import DAG
# from datetime import datetime, timedelta

# dag = DAG(
#     'my_dag',
#     start_date=datetime(2022, 1, 1),
#     schedule_interval='0 0 * * *',
#     catchup=False
# )


In [ ]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine

# Define default_args dict to pass to the DAG
default_args = {
    'owner': 'me',
    'start_date': datetime(2020, 1, 1),
    'depends_on_past': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Create a DAG instance
dag = DAG(
    'etl_dag', default_args=default_args, schedule_interval=timedelta(hours=1)
)

# Define a function to extract data
def extract():
    data = pd.read_csv('data.csv')
    return data

# Define a function to transform data
def transform(**kwargs):
    data = kwargs['data']
    data.fillna(data.mean(), inplace=True)
    return data

# Define a function to load data
def load(**kwargs):
    data = kwargs['data']
    engine = create_engine('sqlite:///data.db')
    data.to_sql('data', engine)

# Create extract operator
extract_task = PythonOperator(
    task_id='extract_task',
    python_callable=extract,
    dag=dag,
)

# Create transform operator
transform_task = PythonOperator(
    task_id='transform_task',
    python_callable=transform,
    provide_context=True,
    dag=dag,
)

# Create load operator
load_task = PythonOperator(
    task_id='load_task',
    python_callable=load,
    provide_context=True,
    dag=dag,
)

# Set task dependencies
extract_task >> transform_task >> load_task


Define tasks: Once the DAG is created, you can define tasks that will be executed as part of the DAG. Airflow provides a wide range of built-in operators for common tasks such as reading from and writing to files, running SQL queries, and more. For example, to read data from a CSV file and load it into a database, you can use the CSVToMySqlOperator and MySqlOperator operators:

In [ ]:
from airflow.operators.mysql_operator import MySqlOperator
from airflow.operators.python_operator import PythonOperator

def read_csv():
    data = pd.read_csv('data.csv')
    return data

read_csv_task = PythonOperator(
    task_id='read_csv',
    python_callable=read_csv,
    dag=dag
)

load_data_task = MySqlOperator(
    task_id='load_data',
    sql='INSERT INTO data (col1, col2, col3) VALUES (:col1, :col2, :col3)',
    mysql_conn_id='mysql_conn_id',
    parameters={'col1': 'val1', 'col2': 'val2', 'col3': 'val3'},
    dag=dag
)


Monitor the DAG: You can monitor the progress of the DAG and view the status of each task using the Airflow web UI. You can also set up email alerts for task failures, and use the Airflow CLI to check the status of the DAG or individual tasks.

Schedule the DAG: Once you've tested the DAG and are satisfied with its performance, you can schedule it to run automatically at regular intervals using the schedule_interval parameter in the DAG definition.

Data Validation: Once the data is loaded, it's important to validate the data to ensure that it's correct, complete and ready for analysis. This could include writing unit tests, running data quality checks, and monitoring data for any errors.

Data Versioning: Data versioning is also important as it allows you to keep track of different versions of the data, and roll back to a previous version if needed. This is especially important if you want to reproduce a previous analysis or if you want to make sure you can always go back to the original data.

Data Backup: It's also important to backup your data regularly, in case of any data loss or corruption. This could include backing up data to a remote server or cloud storage service.